<a href="https://colab.research.google.com/github/reloef/Jobs_Recommender/blob/master/Global_CareerJet_Data_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importing Packages

In [111]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.pipeline import Pipeline
import pickle

import os
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from textblob import TextBlob
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.preprocessing import label_binarize

#lets expand the columns
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.max_columns", None)

#Expanding the rows
pd.set_option('display.max_row', None)

## 2. Loading the data

In [112]:
try:
    from google.colab import files
    uploaded = files.upload()
    import io
    file_data = io.BytesIO(uploaded['jobs_global_title_2022-06-06.csv'])
except:
    file_data = 'jobs_global_title_2022-06-06.csv'

Saving jobs_global_title_2022-06-06.csv to jobs_global_title_2022-06-06 (2).csv


In [113]:
df_1 = pd.read_csv(file_data)
df_1.iloc[0]#.head()

locations               Basel                                                                                                                                                                                                                                                                      
site                    NaN                                                                                                                                                                                                                                                                        
date                    Fri, 03 Jun 2022 22:04:17 GMT                                                                                                                                                                                                                                              
url                     http://jobviewtrack.com/en-us/job-1e4d412c1e5d5b5b11/05573276b7c00cf070f35f9ab144a92a.html?affid=394

## 3. Data Structure

In [114]:
# view data info
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   locations             2970 non-null   object 
 1   site                  0 non-null      float64
 2   date                  2970 non-null   object 
 3   url                   2970 non-null   object 
 4   title                 2970 non-null   object 
 5   description           2970 non-null   object 
 6   company               2872 non-null   object 
 7   salary                276 non-null    object 
 8   salary_min            276 non-null    float64
 9   salary_type           276 non-null    object 
 10  salary_currency_code  276 non-null    object 
 11  salary_max            276 non-null    float64
dtypes: float64(3), object(9)
memory usage: 278.6+ KB


In [115]:
#checking the shape of the data
df_1.shape

(2970, 12)

#### Handling missing values

In [116]:
# Check for the sum and percentage of null values in each column
total_nans = df_1.isnull().sum().sort_values(ascending=False)
percent_nans = (df_1.isnull().mean() * 100).sort_values(ascending=False)
df_nans = pd.concat([total_nans, percent_nans], axis=1, keys=['Total NaN', 'Percent NaN'])
df_nans.head()

,Total NaN,Percent NaN
site,2970,100.000000
salary,2694,90.707071
salary_min,2694,90.707071
salary_type,2694,90.707071
salary_currency_code,2694,90.707071


In [117]:
#create a copy
df_global = df_1.copy()

In [118]:
#checking for nulls
df_global.isna().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970 entries, 0 to 2969
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   locations             2970 non-null   bool 
 1   site                  2970 non-null   bool 
 2   date                  2970 non-null   bool 
 3   url                   2970 non-null   bool 
 4   title                 2970 non-null   bool 
 5   description           2970 non-null   bool 
 6   company               2970 non-null   bool 
 7   salary                2970 non-null   bool 
 8   salary_min            2970 non-null   bool 
 9   salary_type           2970 non-null   bool 
 10  salary_currency_code  2970 non-null   bool 
 11  salary_max            2970 non-null   bool 
dtypes: bool(12)
memory usage: 34.9 KB


In [119]:
#checking the sum of nulls
df_global.isnull().sum()

locations               0   
site                    2970
date                    0   
url                     0   
title                   0   
description             0   
company                 98  
salary                  2694
salary_min              2694
salary_type             2694
salary_currency_code    2694
salary_max              2694
dtype: int64

## 4. Cleaning

In [120]:
# deleting columns with a high percentage and columns with data that won't be useful
df_global.drop(['site', 'salary_type', 'salary_currency_code', 'salary_max'],
        axis = 1, inplace=True)

In [121]:
df_global.head()

,locations,date,url,title,description,company,salary,salary_min
0,Basel,"Fri, 03 Jun 2022 22:04:17 GMT",http://jobviewtrack.com/en-us/job-1e4d412c1e5d5b5b11/05573276b7c00cf070f35f9ab144a92a.html?affid=3943ce450790cc9c0eea91620540ef82,"Commercial Data Analyst - temp. assignment, min 12 months (100%)","internal <b>data</b> <b>scientist</b> team Support global “track and trace"" program as analytics lead for commercial related use cases...For an international company in the global agribusiness market in Basel we are looking for a Commercial <b>Data</b> Analyst...",Adecco,NaN,NaN
1,Geneva,"Wed, 01 Jun 2022 22:28:54 GMT",http://jobviewtrack.com/en-us/job-181e412c19555f5b10/4d919684a2a88bfd3d6bb9310c3f45fb.html?affid=3943ce450790cc9c0eea91620540ef82,Data Scientist,"For one of our clients in Geneva, we are recruiting a <b>Data</b> <b>Scientist</b> for a long term assignment who is open-minded.... Are you looking for you next challenging position? This opportunity is for you! Responsibilities : As a <b>Data</b> <b>Scientist</b> you develop and integrate the health...",Spring Professional,NaN,NaN
2,"Stein, Aargau","Tue, 31 May 2022 22:10:24 GMT",http://jobviewtrack.com/en-us/job-1f1a412c19545f5d15/f6593d6307e4a14343f886b426c5e179.html?affid=3943ce450790cc9c0eea91620540ef82,"Entomology Research Scientist - temporary assignment for 6 months starting September 1st, 2022","Research <b>Scientist</b> - temporary assignment for 6 months starting September 1st, 2022 Your challenges To provide the... of ADME of active ingredients used in crop protection. Plan studies, analyze <b>data</b> and present conclusions in detailed reports...",Adecco,NaN,NaN
3,Europe,"Sat, 04 Jun 2022 07:57:57 GMT",http://jobviewtrack.com/en-us/job-1b1d417e480a060155547300010400130c53006a3a10434e535944171b6c74114e0a07136c3317411a1b191c585f585f2c555f/4883746fd1670a091397d94c9c759ae4.html?affid=3943ce450790cc9c0eea91620540ef82,"senior transporter, senior scientist, applied scientist","smart delivery operation. We are combining cutting-edge mobile and IoT technologies, <b>data</b> streams (video, vehicle telematics... <b>Scientist</b> you will be responsible for building machine learning models and algorithms in areas including mapping and location...",Amazon Europe Core,NaN,NaN
4,Europe,"Sat, 04 Jun 2022 07:54:45 GMT",http://jobviewtrack.com/en-us/job-134f416c4005150149546d020b0907090000380d08014442534a2f290e0d4f1d4e06482d0b06174e1d060e71794e534442164f3d441d450d1c081d13676c11091b1d43455a0d79010c06491b4c0c0f186c2608410e070753794e534442166e5f17/76d255289ad3214e6b58b281879e80ef.html?affid=3943ce450790cc9c0eea91620540ef82,"Senior Applied Scientist, Machine Learning, Amazon Transporter Experience and Technology","smart delivery operation. We are combining cutting-edge mobile and IoT technologies, <b>data</b> streams (video, vehicle telematics... <b>Scientist</b> you will be responsible for building machine learning models and algorithms in areas including mapping and location...",Amazon Europe Core,NaN,NaN


In [122]:
#Analyse the description column
df_global['description'][1]

'For one of our clients in Geneva, we are recruiting a <b>Data</b> <b>Scientist</b> for a long term assignment who is open-minded.... Are you looking for you next challenging position? This opportunity is for you!   Responsibilities :   As a <b>Data</b> <b>Scientist</b> you develop and integrate the health...'

In [123]:
#Analyse the title column
df_global['title'][2]

'Entomology Research Scientist - temporary assignment for 6 months starting September 1st, 2022'

#### Cleaning the decription column

In [124]:
#Define char and print punctuation
char = ['<b>','</b>','–']
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [125]:
#Remove char from df_global['description']
df_global['description'] = df_global['description'].replace(to_replace = char, value="" , regex = True)

#Remove char from df_jobs['title']
df_global['title'] = df_global['title'].replace(to_replace = char, value="" , regex = True)

In [126]:
#Replace / in title with a space
df_global.title = df_global.title.str.replace('/',' ')

#Replace / in description with a space
df_global.description = df_global.description.str.replace('/',' ')

#Replace - in title with a space
df_global.title = df_global.title.str.replace('-',' ')

#Replace - in description with a space
df_global.description = df_global.description.str.replace('-',' ')

In [127]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [128]:
from textblob import Word
nltk.download('stopwords')
nltk.download('wordnet')

#Remove char from df_global['description']
#df_jobs['description'] = df_jobs['description'].replace(to_replace = char, value="" , regex = True)

#Function to remove punctuation
def remove_punctuation(txt):
    return ''.join([l for l in txt if l not in string.punctuation])
df_global['description'] = df_global['description'].apply(remove_punctuation)

#Convert to lowercase
df_global['description'] = df_global['description'].str.lower()

#remove stop words
stop = stopwords.words('english')
df_global['description'] = df_global['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# lemmatization
df_global['description'] = df_global['description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Cleaning of the title column

In [129]:
#Remove char from df_global['title']
#df_jobs['title'] = df_jobs['title'].replace(to_replace = char, value="" , regex = True)

#Function to remove punctuation
def remove_punctuation(txt):
    return ''.join([l for l in txt if l not in string.punctuation])
df_global['title'] = df_global['title'].apply(remove_punctuation)

#Convert to lowercase
df_global['title'] = df_global['title'].str.lower()

#remove stop words
stop = stopwords.words('english')
df_global['title'] = df_global['title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# lemmatization
df_global['title'] = df_global['title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


In [130]:
#Create a column for job id in the dataframe
df_global.insert(0, 'job_id', range(1, 1 + len(df_global)))

## 5. Feature Engineering

### Creating n-grams

In [131]:
#Create columns for ngrams
from nltk.util import ngrams

#Create one gram
n = 1
df_global['one_grams'] = df_global['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['one_grams'].str.len())
   .explode('one_grams')
   .query('count > 0')
)

#Create two grams
n = 2
df_global['two_grams'] = df_global['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['two_grams'].str.len())
   .explode('two_grams')
   .query('count > 0')
)

#Create three grams
n = 3
df_global['three_grams'] = df_global['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['three_grams'].str.len())
   .explode('three_grams')
   .query('count > 0')
)

#Create four grams
n = 4
df_global['four_grams'] = df_global['description'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['four_grams'].str.len())
   .explode('four_grams')
   .query('count > 0')
)

#### Creating the technical Skills column

In [132]:
#list of one gram words we want
words  = ['python','sql','java','aws','architect','powerbi','statistics','pyspark',
          'github','excel','scrum','visualization','docker','kubernetes', 'pandas',
          'mysql', 'postgresql', 'dashboarding', 'hadoop','insights']

#list of two grams words we want
words2 = ['machine learning','data analyst','data analytics','model development','software development','power bi',
          'cloud computing','data science','ms azure','business intelligence','statistical modelling',
          'predictive analytics','microsoft excel','big data', 'data mining', 'data engineering', 'apache hadoop',
         'cloud infrastructure','model algorithm','data collection','hypothesis testing','data gathering']

In [ ]:
#match one gram words
df_global['match_words'] = df_global['one_grams'].apply(lambda terms: [word for word in words 
                                                                if any(word.startswith(term) for term in terms)])

#match two grams words
df_global['match_words2'] = df_global['two_grams'].apply(lambda terms: [word for word in words2 
                                                                if any(word.startswith(term) for term in terms)])

#combine one gram and two grams
df_global['technical skills'] = df_global['match_words']+df_global['match_words2']

In [ ]:
#Dropping columns that won't be needed
df_global.drop(['match_words', 'match_words2'],
        axis = 1, inplace=True)

#### Creating the soft skills column

In [ ]:
#list of one gram words we want
soft1  = ['marketing','writing','creativity','agile','presentation', 'communication', 'curiosity', 'adaptability',
         'kanban', 'research', 'thrive', 'integrity', 'leadership', 'innovative', 'collaborative', 'analysis','probability']

#list of two grams words we want
soft2 = ['team work', 'critical thinking', 'team player', 'problem solving', 'business acumen', 'scrum master',
         'decision making', 'hard working', 'time management']

In [ ]:
#match one gram words
df_global['soft_words'] = df_global['one_grams'].apply(lambda terms: [word for word in soft1 
                                                                if any(word.startswith(term) for term in terms)])

#match two grams words
df_global['soft_words2'] = df_global['two_grams'].apply(lambda terms: [word for word in soft2
                                                                if any(word.startswith(term) for term in terms)])

#combine one gram and two grams
df_global['soft skills'] = df_global['soft_words']+df_global['soft_words2']

#### Creating the qualifications column

In [ ]:
#list of one gram words we want
qua1  = ['engineering', 'degree', 'diploma', 'matric', 'statistics', 'mathematics', 'econometrics',
        'informatics', 'masters', 'postgraduate', 'bachelor', 'honours', 'phd']

#list of two grams words we want
qua2 = ['computer science', 'information technology', 'engineering degree', 'masters degree', 'honours degree',
       'actuarial science', 'data science', 'analytical degrees', 'analytics degree', 'information systems',
       'prostgradaute degree', 'bachelors degree', 'applied mathematics', 'bsc degree', 'business management']

In [ ]:
#match one gram words
df_global['qua_words'] = df_global['one_grams'].apply(lambda terms: [word for word in qua1 
                                                                if any(word.startswith(term) for term in terms)])

#match two grams words
df_global['qua_words2'] = df_global['two_grams'].apply(lambda terms: [word for word in qua2
                                                                if any(word.startswith(term) for term in terms)])

#combine one gram and two grams
df_global['qualification old'] = df_global['qua_words']+df_global['qua_words2']

In [ ]:
#Dropping columns that won't be needed
df_global.drop(['qua_words', 'qua_words2'],
        axis = 1, inplace=True)

#### Creating the experience column

In [ ]:
df_global.head(2000)

In [173]:
#extract all between a numeric and the word experience
df_global['experience'] = df_global['description'].str.extractall(r'([0-9])+(.+?)(experience)').unstack().apply(lambda x:' '.join(x.dropna()), axis=1)
#df_global['experience'] = df_global['experience'].str.extractall(r'([0-9])+(.+?)(year|yr)').unstack().apply(lambda x:' '.join(x.dropna()), axis=1) #uncomment to further filter with the word year/yr in extracted experience
df_global['experience'] = df_global['experience'].fillna('unknown')  #experience string if not found replace with unkwown

#df_global['experience years'] = df_global['experience'].str[:4]
df_global['min experience'] = df_global['experience'].str[:1]
df_global['max experience'] = df_global['experience'].str[3]

df_global['min experience'] = pd.to_numeric(df_global['min experience'], errors='coerce').fillna(0) #minmum years of experience column as a number
df_global['max experience'] = pd.to_numeric(df_global['max experience'], errors='coerce').fillna(0) #maximum years of experience column as a number

#football_soccer_games = sports_games.loc[df_global['experience'].str.contains("year|yr|", case=False)]



df_global[['experience','min experience','max experience']].head(315)

,experience,min experience,max experience
0,unknown,0.0,0.0
1,unknown,0.0,0.0
2,unknown,0.0,0.0
3,unknown,0.0,0.0
4,unknown,0.0,0.0
5,unknown,0.0,0.0
6,unknown,0.0,0.0
7,unknown,0.0,0.0
8,unknown,0.0,0.0
9,unknown,0.0,0.0


#### Creating the contract type/period column

In [ ]:
#Create one gram for title
n = 1
df_global['gram_1'] = df_global['title'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['gram_1'].str.len())
   .explode('gram_1')
   .query('count > 0')
)

#Create two grams
n = 2
df_global['gram_2'] = df_global['title'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
(df_global.assign(count=df_global['gram_2'].str.len())
   .explode('gram_2')
   .query('count > 0')
)

In [ ]:
#list of one gram words we want
cont1  = ['remote', 'training', 'permanent', 'temporary','contract',
          'voluntary', 'learnership', 'internship', 'graduate', 'hybrid']

#list of two grams words we want
cont2 = ['full time', 'part time', 'semi remote', 'graduate program', 'grad program',
          'graduate programs', 'graduate programmes', 'grad programs', 'grad programmes',
         'remote work', 'home office']

In [ ]:
#match one gram words
df_global['contract1'] = df_global['gram_1'].apply(lambda terms: [word for word in cont1 
                                                                if any(word.startswith(term) for term in terms)])
df_global['contract_1'] = df_global['one_grams'].apply(lambda terms: [word for word in cont1 
                                                                if any(word.startswith(term) for term in terms)])

#match two grams words
df_global['contract2'] = df_global['gram_2'].apply(lambda terms: [word for word in cont2
                                                                if any(word.startswith(term) for term in terms)])
df_global['contract_2'] = df_global['two_grams'].apply(lambda terms: [word for word in cont2
                                                                if any(word.startswith(term) for term in terms)])

#combine one gram and two grams
df_global['contract_type/period'] = df_global['contract1']+df_global['contract2']+df_global['contract_1']+df_global['contract_2']


In [ ]:
#Dropping columns that won't be needed
df_global.drop(['one_grams', 'two_grams','three_grams','four_grams','gram_1','gram_2','contract1','contract_1','contract2','contract_2'],
        axis = 1, inplace=True)

In [ ]:
#code for experience
def experience(description):
    #find string between a number and word experience e.g. "2 to 5 years experience"
    try: 
        found_lst = re.findall(r'([0-9])+(.+?)(experience)',description.lower())[0]
        found_lst = "".join(found_lst)
        label = "Found"
    except:
        found_lst = "Unkown"
        label = "Unkown"
    found_lst = found_lst.split(' ')

    if label=="Found":
        if len(found_lst) < 20:
        # labels junior / intermediate / experienced
             if int(found_lst[0]) == 0: # if found has 1st entry to be less than 2 years assign junior
                label = "In-Training"
             elif int(found_lst[0])  <= 2: # if found has 1st entry to be less than 2 years assign junior
                label = "Junior"
             elif 2 < int(found_lst[0])  < 5: # if found has 1st entry to be less than 2 years assign junior
                label = "Intermediate"
             elif int(found_lst[0])  >= 5: # if found has 1st entry to be less than 2 years assign junior
                label = "Senior"
             return " ".join(found_lst),label
        else:
        #add conditions for other experience label
            return 'Too long', label
    elif label =='None':
            return 0, 'graduate role'
df_global["description"].apply(experience)
        
    
    # level descriptor
    #pos_wrds = position.lower().split(' ')
    #pos_lst=['internship', 'trainee', 'grad', 'graduate']
    #position_label=list(set(pos_wrds).intersection(pos_lst))
        
    #return years_label, years_lst, position_label


In [ ]:
df_global.head()

### Separate the words in the lists

In [ ]:
#Separate the skills in technical skills as per job id
df1 = (
    df_global.set_index('job_id')['technical skills'].explode()
    .rename('tech_skills').groupby(level=0).value_counts().reset_index(name='count')
)
#Separate the skills in soft skills as per job id
df2 = (
    df_global.set_index('job_id')['soft skills'].explode()
    .rename('interpersonal_skills').groupby(level=0).value_counts().reset_index(name='count')
)
#Separate the qualifications in qualification old as required by companies
df3 = (
   df_global.set_index('job_id')['qualification old'].explode()
    .rename('qualification').groupby(level=0).value_counts().reset_index(name='count')
)
#Separate the contract tpyes in contract_type/period as per job id
df4 = (
    df_global.set_index('job_id')['contract_type/period'].explode()
    .rename('contract type').groupby(level=0).value_counts().reset_index(name='count')
)

#Dropping columns that won't be needed
df1.drop(['job_id', 'count'], axis = 1, inplace=True)
df2.drop(['job_id', 'count'], axis = 1, inplace=True)
df3.drop(['job_id', 'count'], axis = 1, inplace=True)
df4.drop(['job_id', 'count'], axis = 1, inplace=True)

In [ ]:
df_global = pd.concat([df_global, df1, df2, df3, df4], axis=1, join='outer')

In [ ]:
#Adding the original description column to the new dataframe
df_global['description original'] = df_1['description']

In [ ]:
#saving the dataframe into a csv
df_global.to_csv('df_global_jobs.csv', index = False)

In [ ]:
df_new = pd.read_csv('df_global_jobs.csv')
df_new.head()